In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np

In [2]:
brit_hoops_url = "https://brithoops.com/bbl-player-stats/assists-turnovers.php"
page = requests.get(brit_hoops_url)
soup = BeautifulSoup(page.content)

In [3]:
html = soup.prettify("utf-8")
with open("example-soup.html", "wb") as file:
    file.write(html)

In [4]:
tables = soup.find_all('table')

In [5]:
len(tables)

1

In [6]:
rows = tables[0].find_all('tr')

In [7]:
rows[1]

<tr>
<td class="nm"><a class="pl-profile" href="/bbl-player-stats/profile.php?id=1197">Prentiss Nixon</a></td>
<td>SHE</td>
<td>4</td>
<td>126</td>
<td>31.50</td>
<td>26</td>
<td>6.50</td>
<td>8.26</td>
<td>42.07</td>
<td>10</td>
<td>2.50</td>
<td>3.18</td>
<td>12.46</td>
<td>2.60</td>
</tr>

In [8]:
name = []
team = []
games = []
apg = []
ap40 = []
topg = []
top40 = []

In [9]:
for row in rows[1:]:
    cells = row.find_all('td')
    # The name is held within HTML <a> link tags, hence we used Beautiful Soup to extract only the text part that we want
    name.append(cells[0].find_all('a')[0].get_text())
    team.append(cells[1].get_text())
    games.append(cells[2].get_text())
    apg.append(cells[6].get_text())
    ap40.append(cells[7].get_text())
    topg.append(cells[10].get_text())
    top40.append(cells[11].get_text())

In [10]:
len(name)

98

In [11]:
df = pd.DataFrame(np.column_stack([name, team, games, apg, ap40, topg, top40]), 
                                   columns=["Name", "Team", "Games", "APG", "AP40", "TOPG", "TOP40"])

In [12]:
df

,Name,Team,Games,APG,AP40,TOPG,TOP40
0,Prentiss Nixon,SHE,4,6.50,8.26,2.50,3.18
1,Isaiah Hill,CAL,5,6.40,8.77,2.40,3.29
2,Jalen Ray,SUR,1,6.00,7.58,0.00,0.00
3,Patrick Whelan,CAL,3,5.33,7.25,1.33,1.81
4,Kedrian Johnson,BRI,4,5.25,6.72,3.00,3.84
...,...,...,...,...,...,...,...
93,Cole Long,NEW,2,0.00,0.00,1.50,5.18
94,Totonitoluwa Fagbenle,CAL,5,0.00,0.00,2.40,5.84
95,Greg Wild,CHE,2,0.00,0.00,0.00,0.00
96,Alen Hadzibegovic,LON,1,0.00,0.00,1.00,4.15


In [13]:
df['Games'] = pd.to_numeric(df['Games'])
df = df.loc[df['Games'] > 2].reset_index(drop=True)
len(df)

69

In [14]:
df['AP40'] = pd.to_numeric(df['AP40'])
df['APG'] = pd.to_numeric(df['APG'])
df = df.loc[df['AP40'] > 4.5].reset_index(drop=True)
df = df.loc[df['APG'] > 3].reset_index(drop=True)
len(df)

14

In [15]:
def assists_to_turnovers(assists, turnovers):
    """Computes the assist to turnover ratio as assists / turnovers"""
    ast_turn = assists / turnovers

    return ast_turn

In [16]:
df['TOPG'] = pd.to_numeric(df['TOPG'])
df['AST:TO'] = df.apply(lambda x: assists_to_turnovers(x['APG'], x['TOPG']), axis=1)
df

,Name,Team,Games,APG,AP40,TOPG,TOP40,AST:TO
0,Prentiss Nixon,SHE,4,6.50,8.26,2.50,3.18,2.600000
1,Isaiah Hill,CAL,5,6.40,8.77,2.40,3.29,2.666667
2,Patrick Whelan,CAL,3,5.33,7.25,1.33,1.81,4.007519
3,Kedrian Johnson,BRI,4,5.25,6.72,3.00,3.84,1.750000
4,Cameron Holden,CHE,4,4.75,6.71,2.00,2.83,2.375000
5,Cameron Gooden,SUR,3,4.33,5.46,2.67,3.36,1.621723
6,Jordan Taylor,LON,3,4.00,5.62,2.33,3.28,1.716738
7,Ali Youssouf Ali,MAN,4,4.00,5.64,0.50,0.70,8.000000
8,Nicholas Lewis,MAN,4,3.75,6.40,2.00,3.41,1.875000
9,Jordan Spencer,NEW,5,3.40,5.69,1.20,2.01,2.833333


In [17]:
df['TOP40'] = pd.to_numeric(df['TOP40'])
df['AST40:TO40'] = df.apply(lambda x: assists_to_turnovers(x['AP40'], x['TOP40']), axis=1)

In [18]:
df['AST:TO'].describe()

count    14.000000
mean      2.539596
std       1.728221
min       1.429185
25%       1.625000
50%       1.812500
75%       2.650000
max       8.000000
Name: AST:TO, dtype: float64

In [19]:
df.loc[df['AST:TO'] > 7.9]

,Name,Team,Games,APG,AP40,TOPG,TOP40,AST:TO,AST40:TO40
7,Ali Youssouf Ali,MAN,4,4.0,5.64,0.5,0.7,8.0,8.057143


In [20]:
df['AST40:TO40'].describe()

count    14.000000
mean      2.542614
std       1.742138
min       1.425982
25%       1.625105
50%       1.813416
75%       2.648611
max       8.057143
Name: AST40:TO40, dtype: float64

In [21]:
df_top_10 = df.nlargest(10, 'AST:TO')
df_top_10 = df_top_10.sort_values(by=["AST:TO", "AP40", "TOP40"], ascending=[False, False, True])
df_top_10 = df_top_10[["Name", "Team", "Games", "APG", "AP40", "TOPG", "TOP40", "AST:TO"]]
df_top_10.style.format({'AST:TO':'{:.2}'}).set_caption('SLB Assist to Turnover Leaders').set_table_styles([{
    'selector': 'caption',
    'props': [
        ('text-align', 'left'),
        ('font-size', '18px'),
        ('font-weight', 'bold')
    ]
}]).hide(axis="index")

Name,Team,Games,APG,AP40,TOPG,TOP40,AST:TO
Ali Youssouf Ali,MAN,4,4.000000,5.640000,0.500000,0.700000,8.0
Patrick Whelan,CAL,3,5.330000,7.250000,1.330000,1.810000,4.0
Jordan Spencer,NEW,5,3.400000,5.690000,1.200000,2.010000,2.8
Isaiah Hill,CAL,5,6.400000,8.770000,2.400000,3.290000,2.7
Prentiss Nixon,SHE,4,6.500000,8.260000,2.500000,3.180000,2.6
Cameron Holden,CHE,4,4.750000,6.710000,2.000000,2.830000,2.4
Nicholas Lewis,MAN,4,3.750000,6.400000,2.000000,3.410000,1.9
Kedrian Johnson,BRI,4,5.250000,6.720000,3.000000,3.840000,1.8
Jordan Taylor,LON,3,4.000000,5.620000,2.330000,3.280000,1.7
Ricky Clemons,CHE,4,3.250000,4.860000,2.000000,2.990000,1.6
